In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, make_scorer
from scipy.stats import loguniform

In [3]:
try:
    df_train = pd.read_csv('data/train_processed.csv')
    df_test = pd.read_csv('data/test_processed.csv')
except FileNotFoundError:
    print("FATAL ERROR: Could not find files. Please check file paths.")
    raise

X_train_full = df_train.drop(['has_copd_risk'], axis=1, errors='ignore')
y_train_full = df_train['has_copd_risk']

# Split for Preprocessor fitting (X_train is 80% of data, X_val is 20%)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

# 4. Preprocessor Definition and Fit (Median Imputation + Standard Scaling)
all_features = X_train.columns.tolist()

numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, all_features)
    ],
    remainder='passthrough'
)

# Fit preprocessor on training data and transform it
X_train_processed = preprocessor.fit_transform(X_train)

# 5. Train Logistic Regression Model
log_reg = LogisticRegression(solver='liblinear', random_state=42)
log_reg.fit(X_train_processed, y_train)
print("✅ Logistic Regression Model trained successfully.")

# Optional: Evaluate Baseline F1 Score on Validation Set
X_val_processed = preprocessor.transform(X_val)
y_pred_val = log_reg.predict(X_val_processed)
f1 = f1_score(y_val, y_pred_val)
print(f"Validation F1 Score (Baseline): {f1:.4f}")

# ----------------------------------------------------------------------
# C. PREDICT ON TEST DATA AND SAVE SUBMISSION
# ----------------------------------------------------------------------

# 6. Prepare Test Features
test_ids = pd.read_csv('data/ids.csv')
X_test_full = df_test

# Ensure the columns are in the same order as the training features
X_test_full = X_test_full[all_features]

# 7. Apply Preprocessor to Test Data (Transform only!)
X_test_processed = preprocessor.transform(X_test_full)

# 8. Prediction
y_test_pred = log_reg.predict(X_test_processed)

# 9. Combine IDs and Predictions and Save
submission_df = test_ids.copy()
submission_df['has_copd_risk'] = y_test_pred.astype(int)

submission_df.to_csv('submission/logistic_regression.csv', index=False)

print("\nFirst 5 predictions in submission:")
print(submission_df.head())

✅ Logistic Regression Model trained successfully.
Validation F1 Score (Baseline): 0.6672

First 5 predictions in submission:
   patient_id  has_copd_risk
0       42427              0
1       27412              0
2       19283              1
3       45261              1
4       11155              0


In [2]:
# --- A. LOAD DATA AND SPLIT ---
try:
    df_train = pd.read_csv('data/train_processed.csv')
    df_test = pd.read_csv('data/test_processed.csv')
    test_ids = pd.read_csv('data/ids.csv')
except FileNotFoundError:
    print("FATAL ERROR: Could not find files. Please check file paths ('data/').")
    raise

X_train_full = df_train.drop(['has_copd_risk', 'patient_id'], axis=1, errors='ignore')
y_train_full = df_train['has_copd_risk']

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

# --- B. PREPROCESSING PIPELINE ---
all_features = X_train.columns.tolist()
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer(
    transformers=[('num', numerical_pipeline, all_features)],
    remainder='passthrough'
)
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

# ----------------------------------------------------------------------
# C. HYPERPARAMETER TUNING (Randomized Search)
# ----------------------------------------------------------------------

# 1. Define Model (Base Model uses class_weight for imbalance)
log_reg_base = LogisticRegression(random_state=42, class_weight='balanced', solver='liblinear')

# 2. Define Parameter Grid
param_grid = {
    # C is the inverse of regularization strength. We use a log uniform distribution.
    'C': loguniform(1e-4, 1e4),
    'penalty': ['l1', 'l2'] # L1 performs feature selection, L2 is standard
}

f1_scorer = make_scorer(f1_score)

# 3. Perform Randomized Search CV
log_reg_search = RandomizedSearchCV(
    estimator=log_reg_base,
    param_distributions=param_grid,
    n_iter=30, 
    scoring=f1_scorer,
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

log_reg_search.fit(X_train_processed, y_train)

# 4. Get the Best Model and Score
best_log_reg_model = log_reg_search.best_estimator_
print("\n✅ Logistic Regression Hyperparameter Tuning Complete.")
print(f"Best parameters found: {log_reg_search.best_params_}")

y_pred_val = best_log_reg_model.predict(X_val_processed)
f1_tuned = f1_score(y_val, y_pred_val)
print(f"Validation F1 Score (TUNED LR): {f1_tuned:.4f}")

# ----------------------------------------------------------------------
# D. PREDICT ON TEST DATA AND SAVE SUBMISSION
# ----------------------------------------------------------------------

# 5. Prepare Test Features
X_test_full = df_test.drop('patient_id', axis=1, errors='ignore')
X_test_full = X_test_full[all_features]

# Apply Preprocessor to Test Data
X_test_processed = preprocessor.transform(X_test_full)

# Prediction using the BEST TUNED MODEL
y_test_pred = best_log_reg_model.predict(X_test_processed)

# 6. Combine IDs and Predictions and Save
submission_df = test_ids.copy()
submission_df['has_copd_risk'] = y_test_pred.astype(int)

submission_df.to_csv('submission/tuned_logistic_regression.csv', index=False) 
print("\n✅ Final submission file saved to 'submission/tuned_logistic_regression.csv'")

Fitting 3 folds for each of 30 candidates, totalling 90 fits

✅ Logistic Regression Hyperparameter Tuning Complete.
Best parameters found: {'C': np.float64(0.04673786707787828), 'penalty': 'l2'}
Validation F1 Score (TUNED LR): 0.7047

✅ Final submission file saved to 'submission/tuned_logistic_regression.csv'
